In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas
import gensim

import matplotlib.pyplot as plt
import matplotlib


matplotlib.style.use('ggplot')

## 1. Read data

In [3]:
data_file = "../bdata.20130222.mhci.public.1.txt"

data = pandas.read_csv(data_file, sep = '\t')

In [4]:
data.head()

,species,mhc,peptide_length,cv,sequence,inequality,meas
0,cow,BoLA-HD6,9,TBD,ALFYKDGKL,=,1.000000
1,cow,BoLA-HD6,9,TBD,ALYEKKLAL,=,1.000000
2,cow,BoLA-HD6,9,TBD,AMKDRFQPL,=,4.521706
3,cow,BoLA-HD6,9,TBD,AQRELFFTL,=,1.000000
4,cow,BoLA-HD6,9,TBD,FMKVKFEAL,=,1.576747


In [5]:
print "data shape:", data.shape

data shape: (179692, 7)


In [6]:
print "list of species:", unique(data['species'])

list of species: ['None' 'chimpanzee' 'cow' 'gorilla' 'human' 'macaque' 'mouse']


In [7]:
print "number of unique mhc:", len(unique(data['mhc']))

number of unique mhc: 161


In [8]:
print "number of unique sequences:", len(unique(data['sequence']))

number of unique sequences: 42507


In [9]:
data['meas'].describe()

count    1.796920e+05
mean     2.112472e+04
std      5.986279e+04
min      1.000000e+00
25%      3.589934e+02
50%      1.474869e+04
75%      2.000000e+04
max      1.427660e+07
Name: meas, dtype: float64

 ## 2. Sequences to vector space

In [10]:
s_file = '../data/seqs.txt'

sentences = gensim.models.doc2vec.TaggedLineDocument(s_file)

model = gensim.models.Doc2Vec(sentences, size = 32, window = 4, workers = 4)

In [12]:
df = pandas.DataFrame.from_records(model.docvecs)



In [14]:
data['mhc'] == 'HLA-A*03:01'

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
179662    False
179663    False
179664    False
179665    False
179666    False
179667    False
179668    False
179669    False
179670    False
179671    False
179672    False
179673    False
179674    False
179675    False
179676    False
179677    False
179678    False
179679    False
179680    False
179681    False
179682    False
179683    False
179684    False
179685    False
179686    False
179687    False
179688    False
179689    False
179690    False
179691    False
Name: mhc, dtype: bool